In [2]:
import requests


api_key = '8CTF0D0WVXWCLBVF'

def get_raw_options_data(ticker, date_str):
    url = (f"https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS"
           f"&symbol={ticker}&date={date_str}&apikey={api_key}")
    r = requests.get(url)
    data = r.json()
    print("Raw JSON data for", date_str, ":", data)  # or use st.write(data) if within Streamlit
    return data

# Example usage:
from datetime import date
today_str = date.today().strftime('%Y-%m-%d')
raw_data = get_raw_options_data('AAPL', today_str)


Raw JSON data for 2025-02-11 : {'endpoint': 'Historical Options', 'message': 'success', 'data': [{'contractID': 'AAPL250214C00100000', 'symbol': 'AAPL', 'expiration': '2025-02-14', 'strike': '100.00', 'type': 'call', 'last': '127.90', 'mark': '132.80', 'bid': '132.00', 'bid_size': '130', 'ask': '133.60', 'ask_size': '104', 'volume': '0', 'open_interest': '3', 'date': '2025-02-11', 'implied_volatility': '3.88254', 'delta': '0.99499', 'gamma': '0.00018', 'theta': '-0.20921', 'vega': '0.00305', 'rho': '0.00811'}, {'contractID': 'AAPL250214P00100000', 'symbol': 'AAPL', 'expiration': '2025-02-14', 'strike': '100.00', 'type': 'put', 'last': '0.00', 'mark': '0.01', 'bid': '0.00', 'bid_size': '0', 'ask': '0.01', 'ask_size': '500', 'volume': '0', 'open_interest': '544', 'date': '2025-02-11', 'implied_volatility': '2.98165', 'delta': '-0.00056', 'gamma': '0.00003', 'theta': '-0.02059', 'vega': '0.00041', 'rho': '-0.00001'}, {'contractID': 'AAPL250214C00110000', 'symbol': 'AAPL', 'expiration': '2

In [6]:
import pandas as pd
import requests
from datetime import date

# Replace with your actual Alpha Vantage API key.
ALPHAVANTAGE_API_KEY = '8CTF0D0WVXWCLBVF'

def get_todays_options_data(ticker):
    """
    Retrieves today's options data for the given ticker from Alpha Vantage.
    Returns a DataFrame containing columns such as:
      ['contractID', 'expiration', 'strike', 'type', 'premium', 'volume', 'open_interest', 'data_date']
    """
    today_str = date.today().strftime('%Y-%m-%d')
    url = (
        f"https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS"
        f"&symbol={ticker}&date={today_str}&apikey={ALPHAVANTAGE_API_KEY}"
    )
    r = requests.get(url)
    data = r.json()
    
    # For debugging, you may want to see the raw JSON
    # print("Raw JSON response:", data)
    
    if 'data' in data:
        df = pd.DataFrame(data['data'])
        df['data_date'] = today_str
    else:
        df = pd.DataFrame()
    return df

def display_options_by_expiration(ticker):
    """
    Retrieves today's options data for the ticker, converts the expiration
    column to datetime, groups the options by expiration, and then prints the
    details for each group.
    """
    df = get_todays_options_data(ticker)
    
    if df.empty:
        print("No options data available for today.")
        return
    
    # Ensure numeric columns are converted appropriately.
    df['strike'] = pd.to_numeric(df['strike'], errors='coerce')
    df['last'] = pd.to_numeric(df['last'], errors='coerce')
    df['volume'] = pd.to_numeric(df['volume'], errors='coerce')
    df['open_interest'] = pd.to_numeric(df['open_interest'], errors='coerce')
    
    # Convert the expiration column to datetime.
    df['expiration'] = pd.to_datetime(df['expiration'], errors='coerce')
    
    # Group by expiration date.
    grouped = df.groupby('expiration')
    
    for exp, group in grouped:
        print(f"Expiration: {exp.date() if pd.notnull(exp) else 'Unknown'}")
        # Display selected columns.
        display_cols = ['contractID', 'symbol', 'expiration', 'strike', 'type', 'last', 'volume', 'open_interest']
        print(group[display_cols].head(10))  # show up to 10 rows per expiration
        print("-" * 50)

# Example usage:
ticker = 'AAPL'
df = display_options_by_expiration(ticker)


Expiration: 2025-02-14
            contractID symbol expiration  strike  type    last  volume  \
0  AAPL250214C00100000   AAPL 2025-02-14   100.0  call  127.90       0   
1  AAPL250214P00100000   AAPL 2025-02-14   100.0   put    0.00       0   
2  AAPL250214C00110000   AAPL 2025-02-14   110.0  call    0.00       0   
3  AAPL250214P00110000   AAPL 2025-02-14   110.0   put    0.00       0   
4  AAPL250214C00120000   AAPL 2025-02-14   120.0  call    0.00       0   
5  AAPL250214P00120000   AAPL 2025-02-14   120.0   put    0.00       0   
6  AAPL250214C00130000   AAPL 2025-02-14   130.0  call   98.87       0   
7  AAPL250214P00130000   AAPL 2025-02-14   130.0   put    0.00       0   
8  AAPL250214C00140000   AAPL 2025-02-14   140.0  call   88.52       0   
9  AAPL250214P00140000   AAPL 2025-02-14   140.0   put    0.00       0   

   open_interest  
0              3  
1            544  
2              0  
3              9  
4              0  
5            168  
6             15  
7         

In [16]:
import pandas as pd

# Retrieve today's options data for AAPL.
df = get_todays_options_data('AAPL')

# Check if df is not empty and convert the expiration column to datetime.
if not df.empty:
    df['expiration'] = pd.to_datetime(df['expiration'], errors='coerce')
    # Filter for a specific expiration date
    df_advanced = df[(df['expiration'] == pd.to_datetime('2027-06-17'))]

    print(df_advanced[df_advanced['strike'] == 205])
else:
    print("No options data available.")


Empty DataFrame
Columns: [contractID, symbol, expiration, strike, type, last, mark, bid, bid_size, ask, ask_size, volume, open_interest, date, implied_volatility, delta, gamma, theta, vega, rho, data_date]
Index: []

[0 rows x 21 columns]
